**Importing Libraries**

We are going to examine the No-Show appointments dataset. A few questions that we are examining today are:

1) Which neighbourhood had the highest amount of no-shows?
2)Which neighbourhood had the least amount of no-shows?


Scholarship:

1)Does scholarship have an impact on the no-show
2)What percent of ppl having scholarship do not show up for the appoinmtents
3What percent of ppl not having scholarship do not show up for the appointments


Appointment -scheduled also make it into categorical variable

In [1]:
# Importing all necessary libraries
import pandas as pd
import numpy as np

**Reading Data**

In [2]:
# Reading the Input files
df  = pd.read_csv('../input/KaggleV2-May-2016.csv')

FileNotFoundError: File b'../input/KaggleV2-May-2016.csv' does not exist

**Assessing Data**

In [ ]:
df.info()

In [ ]:
df.shape

So the DataFrame has a total of 110527 rows and 14 columns, also we can see that there are no null values in any columns too.

In [ ]:
df.columns

In [ ]:
sum(df.duplicated())

There are no duplicate values in the dataframe

In [ ]:
df.nunique()

In [ ]:
len(df[df.isnull().any(axis=1)])

In [ ]:
df['Scholarship'].unique()

**Inspecting and Fixing Data Types**

In [ ]:
df.info()

We can see PatientId is a float value, I am converting it to a Integer value as it makes more sense, and i dont expect ID to have decimal point values



In [ ]:
df['PatientId'] =df['PatientId'].astype(int)

In [ ]:
df.sort_values(['PatientId'],inplace=True)

Converting ScheduledDay and AppointmentDay to Datetime objects from string

In [ ]:
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])


In [ ]:
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])

In [ ]:
df.info()

**Exploring Data**

In order to aggregate data for different columns, I am going to change the No-show column values to numeric type.

In [ ]:
df['No-show'].replace('Yes',1, inplace=True)
df['No-show'].replace('No',0,inplace=True)

**Effect of the Gender, Hipertension, Diabetes and Alcoholism Variables**

Now, we are going to aggregate the values in the No-show column for different columns like Gender, Hipertension, diabetes, alcoholism,

In [ ]:
col_names = ['Gender','Hipertension','Diabetes','Alcoholism']

for c in col_names:
   print (df.groupby(c)['No-show'].mean())


So, in the above, we can see that about 3% of people not having Hipertension turned up more than the people having Hipertension 
Also, about 2% of people not having Diabetes, turned up to the appointments more than the ones having Diabetes.

There is very less disparity in the number of people who turned up for the appointments versus the ones who didnot in terms of Gender and Alcoholism.
So, we can safely say, Gender and Alcoholism does not have much of an impact on the No-shows


**Effect of Scholarship variable**

Next, we are going to check, what impact Scholarship has on the number of No-shows. 

1) Do people having Scholarships miss more appointments or the people who donot having Scholarship miss it the most?

In [ ]:
df.groupby('Scholarship')['No-show'].mean()

Ok!! so we can see that the people who are enrolled in the  Brazilain welfare program, did not turn up more  than the ones who did not enroll in the scholarship  by about 4%-5%

People not enrolled in scholarship, who did not showup is about 19% , whereas, people who are enrolled in the scholarship, who did not showup is about 23%


**Effect of SMS_received variable**

Next, we are going to explore,

1) If people who received the SMS to remind them about the appointment, just before the appointment day made it to the appointments, better than the people who were not reminded through SMS

In [ ]:
df.groupby('SMS_received')['No-show'].mean()

Ok! we can see startling numbers here, So almost, people who did not receive SMS turned up better than the people who received the SMS by almost 11%

16% of people who did not receive SMS did not show up for their appointments, but 27% of people who received SMS did not show up for the appointment.

So sending a SMS to remind them about their appointments, tends to have a great effect on the people turn up rate.

**Effect of the Handicap variable**


In [ ]:
df['Handcap'].unique()

In [ ]:
df.groupby('Handcap')['No-show'].mean()

Changing Handcap into a categorical variable

In [ ]:
df['Handcap'] = pd.Categorical(df['Handcap'])
Handicap = pd.get_dummies(df['Handcap'], prefix = 'Handicap')
df = pd.concat([df, Handicap], axis=1)

In [ ]:
h_var = ['Handicap_0','Handicap_1','Handicap_2','Handicap_3','Handicap_4']

for h in h_var:
    print(df.groupby(h)['No-show'].mean())



Here, we can see that the variable Handicap_4 has the highest effect on the number of No-shows.

So people who are disabled in the category Handicap_4 are more likely to not turn up than the ones  in the other categories by almost 13%

**Effect of the Neighbourhood variable**

1) Which neighbourhood seems to have the most number of people missing their appointments?
2) Which neighbourhood has the least number of people missing their appointments?

In [ ]:
df['Neighbourhood_noshows']=  df.groupby(['Neighbourhood'])['No-show'].apply(lambda x: x.cumsum())

In [ ]:
df_max = df.loc[df['Neighbourhood_noshows'] == df['Neighbourhood_noshows'].sort_values().max()]

In [ ]:
df_max['Neighbourhood'].unique()

So the neighbourhood, with the highest number of No-shows is JARDIM CAMBURI with 1465 no-shows


In [ ]:
df_min = (df.loc[df['Neighbourhood_noshows'] == df['Neighbourhood_noshows'].sort_values().min()])

In [ ]:
df_min['Neighbourhood'].nunique()

There are 66 neighbourhoods, which has the minimum number of no-shows which is 0 no-shows and they are as below.

In [ ]:
df_min['Neighbourhood'].unique()

**Effect of Patient Variable**

1) Which patient has the highest number of No-shows?
2) Which patient has the lowest number of No-shows?

In [ ]:
df['Patient_noshows'] = df.groupby('PatientId')['No-show'].apply(lambda x : x.cumsum())

In [ ]:
df.head()

In [ ]:
 df.loc[df['Patient_noshows'] == df['Patient_noshows'].sort_values().max()]

So the Patient with the highest number of No-shows is PatientID : 1421986987763

In [ ]:
df_pa_min =  df.loc[df['Patient_noshows'] == df['Patient_noshows'].sort_values().min()]

In [ ]:
df_pa_min['PatientId'].nunique()

There are 50177 patients who had the minimum number of no-shows which is 0 no-shows so far.

**Effect of the Age Variable**

In [ ]:
df.describe()['Age']

In [ ]:
bin_edges = [0,10,25,50,75,100]
bin_names = ['Under-10','Between_10-25','Between_25-50','Between_50-75','Between_75-100']

df['AgeGroup'] = pd.cut(df['Age'],bin_edges, labels = bin_names)

In [ ]:
df.head()

1)Lets look at what percent of people Under 10 on an average, miss their appointments?

2)Generally, which age-group tends to miss the most apppoinments?

In [ ]:
df['AgeGroup']  = pd.Categorical(df['AgeGroup'])

AgeGrp = pd.get_dummies(df['AgeGroup'], prefix = 'AgeGroup')
df = pd.concat([df, AgeGrp], axis=1)

In [ ]:
df.head()

In [ ]:
age_categories = ['AgeGroup_Under-10','AgeGroup_Between_10-25','AgeGroup_Between_25-50','AgeGroup_Between_50-75','AgeGroup_Between_75-100']
for a in age_categories:
    print(df.groupby(a)['No-show'].mean())

About 20% of the people in the Under_10 category did not show up to their appointments.

So, here we can see that the least turn-up rate is in the Between 50-75 category, which shows that the people in this category turn up much less when compared to people in all other categories. 

**Effect of the Scheduled Day and Appointment Day variables**

In [ ]:
df['Lagtime'] = df['ScheduledDay'].sub(df['AppointmentDay'], axis=0)

In [ ]:
df['Lagtime'] = (df["Lagtime"] / np.timedelta64(1, 'D')).abs()

In [ ]:
df['Lagtime'].min()

In [ ]:
df['Lagtime'].max()

Getting the lagtimes into Groups so that we can check which group has the highest number of no-shows.

Here i am splitting the lagtimes into 6 different columns

In [ ]:
bin_edges_lag = [0,1,7,15,30,100,150,180]
bin_labels_lag = ['Within_a_day','Less_than_7_days','7-15_Days','15-30_Days','30-100_Days','100-150_Days','150-180_Days']

In [ ]:
df['Ltime'] = pd.cut(df['Lagtime'], bin_edges_lag, labels = bin_labels_lag)

In [ ]:
df['Ltime'] = pd.Categorical(df['Ltime'])
LagTime = pd.get_dummies(df['Ltime'], prefix = 'Ltime')
df = pd.concat([df,LagTime], axis=1)

In [ ]:
l_var = ['Ltime_Within_a_day','Ltime_Less_than_7_days','Ltime_7-15_Days','Ltime_15-30_Days','Ltime_30-100_Days','Ltime_100-150_Days','Ltime_150-180_Days']
for l in l_var:
    print(df.groupby(l)['No-show'].mean())

In [ ]:
df.columns

In [ ]:
df = df.drop(['Neighbourhood','Handcap','PatientId','AppointmentID','ScheduledDay','AppointmentDay'], axis=1)

In [ ]:
df.columns

**Visualizations:**

In [ ]:
l_var = ['Ltime_Within_a_day','Ltime_Less_than_7_days','Ltime_7-15_Days','Ltime_15-30_Days','Ltime_30-100_Days','Ltime_100-150_Days','Ltime_150-180_Days']
firstval = list()
for l in l_var:
    firstval.append(df.groupby(l)['No-show'].mean()[0])
firstval = tuple(firstval)

In [ ]:
l_var = ['Ltime_Within_a_day','Ltime_Less_than_7_days','Ltime_7-15_Days','Ltime_15-30_Days','Ltime_30-100_Days','Ltime_100-150_Days','Ltime_150-180_Days']
secondval = list()
for l in l_var:
    secondval.append(df.groupby(l)['No-show'].mean()[1])
secondval = tuple(secondval)

**Visualizations**

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
% matplotlib inline

**Lagtime variable**

In [ ]:
N = 7
ind = np.arange(N)
width = 0.35
fig,ax = plt.subplots()

rects1 = ax.bar(ind, firstval, width, color='y')
rects2 = ax.bar(ind+width,secondval, width, color='r' )

ax.set_ylabel('No-show Rate')
ax.set_title('No-Shows based on LagTime')
ax.set_xticks(ind + width / 2)
ax.set_xticklabels(('Ltime_Within_a_day','Ltime_Less_than_7_days', 'Ltime_7-15_Days', 'Ltime_15-30_Days', 'Ltime_30-100_Days', 'Ltime_100-150_Days','Ltime_150-180_Days'), rotation='vertical')

ax.legend((rects1[0], rects2[0]), ('Not In', 'In'))
plt.show()

Based on this graph, we can see that the people who scheduled the appointment on the same day, were most likelier to show up than the people who booked prior to their appointment day

The number of people who show up seems to comparitively decrease as the number of days go up and after about a 100 days it has gone up very slightly.


**Handicap**

In [ ]:
h_var = ['Handicap_0','Handicap_1','Handicap_2','Handicap_3','Handicap_4']
h_showup = list()
h_noshow = list()
for h in h_var:
    h_showup.append(df.groupby(h)['No-show'].mean()[0])
    h_noshow.append(df.groupby(h)['No-show'].mean()[1])
tuple(h_showup)
tuple(h_noshow)



In [ ]:
N = 5
ind = np.arange(N)
width = 0.35
fig,ax = plt.subplots()

rects1 = ax.bar(ind, h_showup, width, color='y')
rects2 = ax.bar(ind+width,h_noshow, width, color='r' )

ax.set_ylabel('No-show Rate')
ax.set_title('Average Handicap No-show Rate')
ax.set_xticks(ind + width / 2)
ax.set_xticklabels(('Handicap_0','Handicap_1', 'Handicap_2', 'Handicap_3', 'Handicap_4'), rotation='vertical')

ax.legend((rects1[0], rects2[0]), ('Not Having', 'Having'))
plt.show()

Based on the graph above, we can see that,  people in the  handicap_4 category are most likely to not turn up in comparision to the other categories.

**AgeGroup**

In [ ]:
df['Age'].plot(kind='hist')

In [ ]:
a_var = ['AgeGroup_Under-10','AgeGroup_Between_10-25','AgeGroup_Between_25-50','AgeGroup_Between_50-75','AgeGroup_Between_75-100']

a_showup = list()
a_noshow = list()
for a in a_var:
    a_showup.append(df.groupby(a)['No-show'].mean()[0])
    a_noshow.append(df.groupby(a)['No-show'].mean()[1])
tuple(a_showup)
tuple(a_noshow)




In [ ]:
N = 5
ind = np.arange(N)
width = 0.35
fig,ax = plt.subplots()

rects1 = ax.bar(ind, h_showup, width, color='y')
rects2 = ax.bar(ind+width,h_noshow, width, color='r' )

ax.set_ylabel('No-show Rate')
ax.set_title('Average Handicap No-show Rate')
ax.set_xticks(ind + width / 2)
ax.set_xticklabels(('AgeGroup_Under-10','AgeGroup_Between_10-25','AgeGroup_Between_25-50','AgeGroup_Between_50-75','AgeGroup_Between_75-100'), rotation='vertical')

ax.legend((rects1[0], rects2[0]), ('Not In', 'In'))
plt.show()

**SMS received**

In [ ]:
df['SMS_received'].replace(0,'Not Received', inplace=True)
df['SMS_received'].replace(1,'Received',inplace=True)

In [ ]:
df.groupby('SMS_received')['No-show'].mean().plot(kind='bar')

**Scholarship**

In [ ]:
df['Scholarship'].replace(0,'Not Enrolled',inplace=True)
df['Scholarship'].replace(1,'Enrolled',inplace=True)

In [ ]:
df.groupby('Scholarship')['No-show'].mean().plot(kind='bar')

**Diabetes, Alcoholism, Hipertension**

In [ ]:
n_var = ['Diabetes','Alcoholism','Hipertension']
n_showup = list()
n_noshow = list()
for n in n_var:
    n_showup.append(df.groupby(n)['No-show'].mean()[0])
    n_noshow.append(df.groupby(n)['No-show'].mean()[1])
tuple(n_showup)
tuple(n_noshow)




In [ ]:
N = 3
ind = np.arange(N)
width = 0.35
fig,ax = plt.subplots()

rects1 = ax.bar(ind, n_showup, width, color='y')
rects2 = ax.bar(ind+width,n_noshow, width, color='r' )

ax.set_ylabel('No-show Rate')
ax.set_title('Average No-show Rate across different categories')
ax.set_xticks(ind + width / 2)
ax.set_xticklabels(( 'Diabetes','Alcoholism', 'Hipertension'), rotation='vertical')

ax.legend((rects1[0], rects2[0]), ('Not Having', 'Having'))
plt.show()

**Gender**

In [ ]:
df.groupby('Gender')['No-show'].mean().plot(kind='bar')

Based on the graph above, we can see that the gender does not play much of an important rule in determining who will miss the most appointments, as both males and females have missed about 20% of their appointments.

**Conclusion**

So finally, based on the graphs and analysis above, we can see that the variables, Gender,Diabetes, Alcoholism donot play much of an important role in determining whether a patient is likely to turn up to their future appointments or not

However, the variables:

Handicap
Age
Lagtime
Scholarship
SMS_received

do seem to play a critical role.

People in Handicap_4 category seem to miss most of their appointments in comparision to the people in the other handicap categories on the whole.

People in the Age Group 75-100 also seem to miss almost 10% of their appoinments more than the other people

When it comes to the lagtime, people who booked the appointment the same day seem to make it to the appointment much better than the people who booked their appointments earlier by almost 20%

People who did not receive SMS turned up better than the people who received the SMS by almost 11%

People who have scholarship turned lesser than the people who do not have scholarship by almost 4%